In [212]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [213]:
df = pd.read_csv("data/p1ch4/tabular-wine/winequality-white.csv", sep=";")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [214]:
input_columns = df.columns[:11]
output_column = df.columns[11]

In [215]:
input_df = df[input_columns]
output_df = df[output_column]

In [216]:
input_np = input_df.to_numpy(dtype=np.float32)
input_t = torch.tensor(input_np)

In [217]:
output_np = output_df.to_numpy(dtype=np.float32)
output_t = torch.tensor(output_np).unsqueeze(1)

In [218]:
# Training Validation Split
batch = input_t.size(0)

shuffled_indices = torch.randperm(batch)
training_indices = shuffled_indices[:int(batch * 0.8)]
validation_indices = shuffled_indices[int(batch * 0.8):]

training_input_t = input_t[training_indices]
training_output_t = output_t[training_indices]

validation_input_t = input_t[validation_indices]
validation_output_t = output_t[validation_indices]

In [219]:
# Input Normalisation
mean_training = torch.mean(training_input_t, dim=0)
sd_training = torch.std(training_input_t, dim=0)

training_input_t = (training_input_t - mean_training) / sd_training
validation_input_t = (validation_input_t - mean_training) / sd_training

In [220]:
seq_model = nn.Sequential(
  nn.Linear(11, 110),
  nn.Tanh(),
  nn.Linear(110, 1)
)

In [221]:
learning_rate = 1e-2
optimizer = optim.SGD(seq_model.parameters(), learning_rate)

In [222]:
def training_loop(epochs: int, model: nn.Sequential, optimizer: optim.SGD, loss_function: nn.MSELoss, training_input: torch.Tensor, training_output: torch.Tensor,
                  validation_input: torch.Tensor, validation_output: torch.Tensor) -> nn.Sequential:
  for epoch in range(1, epochs + 1):
    training_pred_output = model(training_input)
    training_loss = loss_function(training_pred_output, training_output)

    with torch.no_grad():
      validation_pred_output = model(validation_input)
      validation_loss = loss_function(validation_pred_output, validation_output)

    optimizer.zero_grad()
    training_loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
      print(f"Epoch: {epoch}, Training Loss: {training_loss}, Validation Loss: {validation_loss}")

  return model

In [223]:
trained_model = training_loop(10000, seq_model, optimizer, nn.MSELoss(), training_input_t, training_output_t, validation_input_t, validation_output_t)

Epoch: 1000, Training Loss: 0.5323497653007507, Validation Loss: 0.5697637796401978
Epoch: 2000, Training Loss: 0.5053613781929016, Validation Loss: 0.5491212606430054
Epoch: 3000, Training Loss: 0.4904978573322296, Validation Loss: 0.5404382944107056
Epoch: 4000, Training Loss: 0.4793701469898224, Validation Loss: 0.5350012183189392
Epoch: 5000, Training Loss: 0.46995288133621216, Validation Loss: 0.5299116969108582
Epoch: 6000, Training Loss: 0.46163731813430786, Validation Loss: 0.5245810151100159
Epoch: 7000, Training Loss: 0.454285204410553, Validation Loss: 0.5198075771331787
Epoch: 8000, Training Loss: 0.44785434007644653, Validation Loss: 0.5167072415351868
Epoch: 9000, Training Loss: 0.4422423839569092, Validation Loss: 0.5147786736488342
Epoch: 10000, Training Loss: 0.43728259205818176, Validation Loss: 0.5130859613418579


In [225]:
pred_score = trained_model(validation_input_t)
pred_score

tensor([[5.1557],
        [6.2582],
        [6.3388],
        [5.9426],
        [5.7144],
        [5.9359],
        [6.0159],
        [6.4155],
        [5.6961],
        [6.6106],
        [5.6631],
        [5.8628],
        [5.4715],
        [6.8394],
        [6.3841],
        [6.5147],
        [6.3324],
        [5.5824],
        [4.3612],
        [6.4095],
        [5.2437],
        [6.3768],
        [5.4781],
        [5.9403],
        [5.6662],
        [5.9403],
        [6.0378],
        [5.6904],
        [5.3684],
        [6.5668],
        [6.2197],
        [6.6306],
        [5.0473],
        [5.2755],
        [5.5471],
        [5.8528],
        [5.2981],
        [6.6665],
        [6.2894],
        [5.6187],
        [4.8618],
        [6.6827],
        [5.4280],
        [5.8673],
        [6.6355],
        [5.6225],
        [6.5427],
        [5.2355],
        [5.8455],
        [6.7425],
        [5.1929],
        [5.4296],
        [5.0258],
        [6.0617],
        [6.1771],
        [5